<a href="https://colab.research.google.com/github/sayarghoshroy/Summaformers/blob/main/laySumm_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import pickle as pkl
!pip install rouge_score
from rouge_score import rouge_scorer
!pip install transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import GPT2Model, GPT2Config, GPT2Tokenizer, GPT2Tokenizer, GPT2LMHeadModel
from transformers import BartTokenizer, BartForConditionalGeneration
import numpy as np
from tqdm import tqdm_notebook

import os
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
filename = open("/content/drive/My Drive/Scientific_Doc_Summarization_Raw_Data/Pickled Datasets/laySumm_Train.pkl",'rb')
data = pkl.load(filename)

print("Number of Datapoints:", len(data))
print(data[0]['laysumm']['laysumm'])

Number of Datapoints: 572
['PARAGRAPH', 'Neural networks have shown promising capabilities in dealing with nonlinear systems.', 'Combined with a dynamic part, they can learn and identify complicated systems.', 'On the other hand, fractional order differentiation has provided more degrees of freedom and therefore more accuracy for exploring the behaviour of dynamical systems.', 'This paper introduces a fractional order neural network capable of identifying complicated systems.', 'Input-output representational capacity is shown, learning rules for the parameters are derived and its performance on the real data of a 2.5MW wind turbine is examined.', 'The results are amazing and the method can be used for other industrial systems.']


In [ ]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

word_counts = []
for i in range(572):
  words = 0
  for sent in data[i]['laysumm']['laysumm']:
    tokens = tokenizer.tokenize(sent)
    words += len(tokens)
  word_counts.append(words)

print("Average Number of Words:")
print(np.mean(np.asarray(word_counts)))

Average Number of Words:
86.3548951048951


### Get Dataset

In [ ]:
with open('/content/drive/My Drive/Scientific_Doc_Summarization_Raw_Data/trainids.pkl','rb') as f:
    train_ids = pkl.load(f)
    
with open('/content/drive/My Drive/Scientific_Doc_Summarization_Raw_Data/testids.pkl','rb') as f:
    test_ids = pkl.load(f)

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [ ]:
print(device)

cuda


### Initialize Dataset Loader for training


In [ ]:
def getRawDoc(doc):
    rawdoc = ''
    
    for item in doc:
        if type(item) == list:
            rawdoc += ''.join(item[1:]) + ' '
        else:
            continue
            
    return rawdoc

In [ ]:
import nltk
nltk.download('punkt')

def get_text(units):
    text = ""

    # Only considering the first and the last paragraphs

    count_pg = 0
    pg_id = 0

    for sent in units:
        if sent == 'PARAGRAPH':
            count_pg += 1
    
    for sent in units:
        if sent == 'PARAGRAPH':
            pg_id += 1
            continue
        
        if pg_id != 1 and pg_id != 2 and pg_id != count_pg:
          continue
        
        tokens = nltk.word_tokenize(sent)
        
        for token in tokens:
            text += token.strip().rstrip("\n") + " "
            
    return text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt', truncation=True)
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt', truncation=True)

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [ ]:
getRawDoc(list(data[0]['abstract'].values())).replace("\n"," ").replace(".", " . ").replace(",", " ,").replace("(", "( ").replace(")", " )")

'In this paper , a Variable-Order Fractional Single-layer Neural Network ( VOFSNN ) and a Variable-Order Fractional Multi-layer Neural Network ( VOFMNN ) are proposed to identify nonlinear systems assuming all the system states are measurable . Fractional Lyapunov-like approach and Gronwall–Bellman integral inequality are employed to prove stability and asymptotic stability conditions of the identification error dynamics . A set of novel stable learning rules for the fractional order , the hidden layer weights and the output layer weights are derived to update the proposed VOFSNN and VOFMNN parameters . The proposed methods capabilities are evaluated and confirmed by the practical data gathered from a wind turbine under operation in a wind farm .  '

In [ ]:
get_text(data[0]['laysumm']['laysumm']).replace("\n"," ")

'Neural networks have shown promising capabilities in dealing with nonlinear systems . Combined with a dynamic part , they can learn and identify complicated systems . On the other hand , fractional order differentiation has provided more degrees of freedom and therefore more accuracy for exploring the behaviour of dynamical systems . This paper introduces a fractional order neural network capable of identifying complicated systems . Input-output representational capacity is shown , learning rules for the parameters are derived and its performance on the real data of a 2.5MW wind turbine is examined . The results are amazing and the method can be used for other industrial systems . '

In [ ]:
get_text(data[0]['fulltext']['conclusion']).replace("\n"," ")

'This paper proposes two neuro-identifiers , namely , the VOFSNN and the VOFMNN , to identify a big enough class of fractional order nonlinear dynamic systems dealing with complex phenomena . Compared with the FONN structure used in the literature , the proposed identifier models various systems more accurate due to its rich structure . In addition , the fractional order is also updated based on a new proposed adaptation law . To prove the stability of the proposed algorithm , first , we give two Lyapunov functions . Then , by using fractional order differential equation analysis , Gronwall–Bellman inequality and some useful Lemmas , the stability conditions are determined . The new derived learning rules guarantee that the identification error and the parameters of identifiers remain bounded . To make a comparison between two proposed identifiers with IONN , the introduced algorithms are employed to identify a wind turbine using experimental data . The effectiveness of the proposed al

In [ ]:
# summaries = [getRawDoc(list(data['abstract'].values())).replace("\n"," ")]
summaries = {}
text = {}
count_blank = 0
intros = 0
missing_abs_ids = []

for x in data:
    doc = get_text(x['abstract']['abstract']).replace("\n"," ")

    if (doc.strip() == ""):
      missing_abs_ids.append(x['id'])
      count_blank += 1
    
    if 'conclusion' in x['fulltext'].keys():
      doc_conc = get_text(x['fulltext']['conclusion']).replace("\n"," ").replace("(", "( ").replace(")", " )")
    elif 'conclusions' in x['fulltext'].keys():
      doc_conc = get_text(x['fulltext']['conclusions']).replace("\n"," ").replace("(", "( ").replace(")", " )")
    else:
      doc_conc = ""

    if 'introduction' in x['fulltext'].keys():
      intros += 1
      doc_intro = get_text(x['fulltext']['introduction']).replace("\n"," ")

    else:
      doc_intro = ""

    doc = doc + " " + doc_intro + " " + doc_conc 
    
    text[x['id']] = doc.strip()
    
    summaries[x['id']] = get_text(x['laysumm']['laysumm'])

In [ ]:
train_dataset = pd.DataFrame([[summaries[x],text[x]] for x in train_ids], columns = ['text', 'ctext']) 
val_dataset = pd.DataFrame([[summaries[x],text[x]] for x in test_ids], columns = ['text', 'ctext']) 

### Initialize Model Instance

In [ ]:
modelname = 'bart'

In [ ]:
if modelname == 't5':
    tokenizer = T5Tokenizer.from_pretrained('t5-base')
    model = T5ForConditionalGeneration.from_pretrained('t5-base').to(device)

if modelname == 'gpt2':
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)
    tokenizer.pad_token = tokenizer.eos_token
    
if modelname == 'bart':
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-xsum')
    model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-xsum').to(device)
    # model_statedict_path = "drive/My Drive/Scientific_Doc_Summarization_Raw_Data/Saved Models/BEST_bart_abs2sum_three_tune.pt"
    # model.load_state_dict(torch.load(model_statedict_path))

Some weights of BartForConditionalGeneration were not initialized from the model checkpoint at facebook/bart-large-cnn and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, lm_labels=lm_labels)
        loss = outputs[0]
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=180, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%25==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [ ]:
TRAIN_BATCH_SIZE = 8    # input batch size for training (default: 64)
VALID_BATCH_SIZE = 16    # input batch size for testing (default: 1000)
TRAIN_EPOCHS = 1000       # number of epochs to train (default: 10)
VAL_EPOCHS = 1 
LEARNING_RATE = 5 * 1e-5    # learning rate (default: 0.01)
SEED = 42               # random seed (default: 42)
MAX_LEN = 200
SUMMARY_LEN = 150

scale = 1

In [ ]:
training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)
val_set = CustomDataset(val_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)

In [ ]:
train_params = {
        'batch_size': TRAIN_BATCH_SIZE,
        'shuffle': True,
        'num_workers': 0
        }

val_params = {
    'batch_size': VALID_BATCH_SIZE,
    'shuffle': False,
    'num_workers': 0
    }

training_loader = DataLoader(training_set, **train_params)
val_loader = DataLoader(val_set, **val_params)

# Model declared above
# model_statedict_path = "drive/My Drive/Scientific_Doc_Summarization_Raw_Data/bart_gap_best.pt"
# model.load_state_dict(torch.load(model_statedict_path))

# Defining the optimizer that will be used to tune the weights of the network in the training session. 
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

print('Initiating Fine-Tuning for the model on our dataset')

old_best = 0

# for early stopping

for epoch in range(TRAIN_EPOCHS):
    print("Epoch Number:", epoch)
    print("Learn Rate:", LEARNING_RATE)
    
    for val_epoch in range(VAL_EPOCHS):
        predictions, actuals = validate(val_epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
        final_df.to_csv('./predictions.csv')
        print('Output Files generated for review')

    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2','rougeL'])
    # scores = scorer.score(output,refsum)

    final_df['Actual Text'] = final_df['Actual Text'].apply(lambda x: x.lower())
    final_df['r1'] = final_df.apply(lambda x: scorer.score(x['Generated Text'], x['Actual Text'])['rouge1'][2], axis=1)
    final_df['r2'] = final_df.apply(lambda x: scorer.score(x['Generated Text'], x['Actual Text'])['rouge2'][2], axis=1)
    final_df['rl'] = final_df.apply(lambda x: scorer.score(x['Generated Text'], x['Actual Text'])['rougeL'][2], axis=1)

    print(final_df['r1'].to_numpy().mean())
    print(final_df['r2'].to_numpy().mean())
    print(final_df['rl'].to_numpy().mean())

    if (final_df['r1'].to_numpy().mean() > old_best):
      print("Continuing on track")
      old_best = final_df['r1'].to_numpy().mean()

      model_save_name = 'bart_intco.pt'
      path = F"/content/drive/My Drive/Scientific_Doc_Summarization_Raw_Data/{model_save_name}"

      delete_filename = path
      open(delete_filename, 'w').close() # overwrite and make the file blank instead - ref: https://stackoverflow.com/a/4914288/3553367
      os.remove(delete_filename) # delete the blank file from google drive will move the file to bin instead

      model_save_name = 'bart_intco.pt'
      path = F"/content/drive/My Drive/Scientific_Doc_Summarization_Raw_Data/{model_save_name}" 
      torch.save(model.state_dict(), path)

    else:
      print("Backtrack")
      model_statedict_path = "drive/My Drive/Scientific_Doc_Summarization_Raw_Data/bart_intco.pt"
      model.load_state_dict(torch.load(model_statedict_path))
      LEARNING_RATE /= (4 * scale)
      scale *= 4
      if LEARNING_RATE < 1e-7:
        break
      optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

    train(epoch, tokenizer, model, device, training_loader, optimizer)

In [ ]:
# Thank You ^_^